## How To Perform an Amplitude Rabi Experiment

### Prerequisites
This guide assumes you have a configured `DeviceSetup` as well as `Qubit` objects with assigned parameters. Please see these guides (add links) if you need to create your setup and qubits for the first time. However, you can also run this notebook "as is" using an emulated session. If you are just getting started with the LabOne Q Applications Library, please don't hesitate to reach out to us at info@zhinst.com.

### Background
In this how-to guide, you'll perform a measurement to determine the qubit transition frequency. The LabOne Q Applications Library provides different experiment workflows to achieve this task. You can either just sweep the frequency of a qubit drive pulse using the `qubit_spectroscopy` experiment workflow, or perform a 2D sweep to optimize at the same time the amplitude of the qubit drive pulse (the `qubit_spectroscopy_amplitude` experiment workflow).

Below is a simple diagram of showing the ground and the first excited state of a qubit, where the transition frequency corresponds to the $\omega_{01}$.

![](../images/qubit_ge.svg "Sketch of ground and first excited states of a qubit")

As the swept qubit drive frequency comes close to the qubit transition frequency, the qubit will be driven out of its ground state. Due to dispersive coupling (see dispersive readout how-to for more detail), the signal from the readout resonator will then experience a shift in magnitude and phase.

Plotting the readout signal, we can obtain a lorentzian lineshape as a function of the qubit drive frequency. This lorentzian will be centered around the qubit transition frequency, and from its linewidth we can approximate the qubit dephasing time. Below is an example dataplot of a qubit spectroscopy measurement:

![](../images/qubit_spec_result.png "Example data plot of qubit spectroscopy")

### Imports

You'll start by importing the qubit spectroscopy experiment from `laboneq_applications`, as well as `laboneq.simple` and a demo QPU and device setup to run in emulation mode.

In [1]:
import numpy as np
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation
from laboneq.simple import *

from laboneq_applications.experiments import (
    qubit_spectroscopy,
    qubit_spectroscopy_amplitude,
)
from laboneq_applications.qpu_types.tunable_transmon.demo_qpus import demo_platform

### QPU and Device Setup

You'll generate six qubits with pre-defined parameters, as well as a `Device_Setup` consisting of a SHFQC+, HDAWG, and PQSC. If you already have your own `DeviceSetup` and qubits configured, you'll instead initialize the session using your setup.

In [2]:
my_platform = demo_platform(6)

Then, you'll connect to the `Session`. Here we connect to an emulated one:

In [ ]:
session = Session(my_platform.setup)
session.connect(do_emulation=True)

### Running the Qubit Spectroscopy Workflow

First, make a demo qubit, and choose a frequency to sweep within +/-500MHz of the drive_lo frequencies of the qubits. Let's choose a realistic value of 6GHz.

In [ ]:
# our qubits live here in the demo setup:
qubits = my_platform.qpu.qubits

for q in [qubits[0], qubits[1]]:
    print("LO value:",q.parameters.drive_lo_frequency/1e9, "GHz")
    q.parameters.drive_lo_frequency = 6e9
    print("updated LO value:",q.parameters.drive_lo_frequency/1e9, "GHz")

You'll now make the experiment workflow and run:

In [ ]:
my_workflow = qubit_spectroscopy.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    frequencies=[np.linspace(5.8e9, 6.2e9, 101), np.linspace(5.8e9, 6.2e9, 101)],
)

my_results = my_workflow.run()

#### Output Simulation

You can also inspect the compiled experiment and plot the simulated output:

In [ ]:
compiled_qubit_spec = my_results.tasks["compile_experiment"].output
plot_simulation(compiled_qubit_spec, length=50e-6)

#### Inspecting the Source Code

You can inspect the source code of the `create_experiment` task defined in `qubit_spectroscopy` to see how the experiment pulse sequence is created:

In [ ]:
print(qubit_spectroscopy.create_experiment.src)

### Changing the Options

We can give our Qubit Spectroscopy experiment options. First, inspect what they currently are:

In [ ]:
my_new_opts = qubit_spectroscopy.experiment_workflow.options()
my_new_opts

Then provide new options. Let's change the counts

In [9]:
my_new_opts.count(1)

#### Run the workflow with updated options

Now, run the workflow with new options and inspect the simulated output. Here, we also reduce the frequency sweep to 2 points, and length of the coherent drive to 1us. Since the new experiment count is 1, you'll notice that there is now only 2 sequences, corresponding to the 2 frequency sweep points repeated once.

In [ ]:
my_new_workflow = qubit_spectroscopy.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    frequencies=[np.linspace(5.8e9, 6.2e9, 101), np.linspace(5.8e9, 6.2e9, 101)],
    options=my_new_opts,
)

my_new_results = my_new_workflow.run()
new_compiled_qubit_spec = my_new_results.tasks["compile_experiment"].output
plot_simulation(new_compiled_qubit_spec, length=50e-6)

### Temporary settings

The qubit parameters are used to control the settings of pulses and instruments during the experiment. We can run the qubit spectroscopy experiment with different settings by passing it a copy of the qubits with modified parameters. Below example modifies the reset_delay_length, spectroscopy_length, and spectroscopy_amplitude of the qubits.

In [ ]:
# Make a copy of the qubits
temp_qubits = my_platform.qpu.copy_qubits()
# Change the reset delay length of the drive pulses
temp_qubits[0].parameters.reset_delay_length = 10e-6
temp_qubits[1].parameters.reset_delay_length = 10e-6
temp_qubits[0].parameters.spectroscopy_length = 2e-6
temp_qubits[1].parameters.spectroscopy_length = 2e-6
temp_qubits[0].parameters.spectroscopy_amplitude = 0.5

my_new_workflow = qubit_spectroscopy.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[temp_qubits[0], temp_qubits[1]],  # pass temporary qubits
    frequencies=[np.linspace(5.8e9, 6.2e9, 101), np.linspace(5.8e9, 6.2e9, 101)],
    options=my_new_opts,
)

my_new_results = my_new_workflow.run()
new_compiled_qubit_spec = my_new_results.tasks["compile_experiment"].output
plot_simulation(new_compiled_qubit_spec, length=50e-6)

# revert the reset delay length of the drive pulses back to its default value of 1us
temp_qubits[0].parameters.reset_delay_length = 1e-6
temp_qubits[1].parameters.reset_delay_length = 1e-6
temp_qubits[0].parameters.spectroscopy_length = 5e-6
temp_qubits[1].parameters.spectroscopy_length = 5e-6
temp_qubits[0].parameters.spectroscopy_amplitude = 1

### Sweep the amplitude at the same time

Getting back to our found parameters, we now repeat the workflow while sweeping the amplitude of the drive frequency at the same time. Since the analysis and the interface are different, a dedicated workflow is used, but the input only differs slightly. 

Since the amplitude is swept in the outer loop, while frequency is swpet in the inner loop, we set the frequency sweep to only 2 to have the amplitude modification visible in the OutputSimulator.

In [ ]:
my_workflow = qubit_spectroscopy_amplitude.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[temp_qubits[0], temp_qubits[1]],  # pass temporary qubits
    amplitudes = [np.linspace(0.1, 0.5, 2), np.linspace(0.1, 0.5, 2)],
    frequencies=[np.linspace(5.8e9, 6.2e9, 101), np.linspace(5.8e9, 6.2e9, 101)],
)

my_results = my_workflow.run()
compiled_qubit_spec = my_results.tasks["compile_experiment"].output
plot_simulation(compiled_qubit_spec, length=50e-6)

Great! You've now run your Qubit Spectroscopy experiment. Check out these other experiments to keep characterizing your qubits:

In [13]:
# TODO: Add experiment links
# TODO: Add Analysis